In [1]:
import torch
torch.cuda.get_device_properties(0) 

_CudaDeviceProperties(name='NVIDIA GeForce RTX 4090', major=8, minor=9, total_memory=24089MB, multi_processor_count=128, uuid=68ce6bda-4517-9a6c-268d-51bbda3ffa99, L2_cache_size=72MB)

In [2]:
from transformers import AutoModelForCausalLM

model_name = "Qwen/Qwen2.5-7B-Instruct-AWQ"

model = AutoModelForCausalLM.from_pretrained(model_name, 
                                             device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
model.cuda()

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): WQLinear_GEMM(in_features=3584, out_features=3584, bias=True, w_bit=4, group_size=128)
          (k_proj): WQLinear_GEMM(in_features=3584, out_features=512, bias=True, w_bit=4, group_size=128)
          (v_proj): WQLinear_GEMM(in_features=3584, out_features=512, bias=True, w_bit=4, group_size=128)
          (o_proj): WQLinear_GEMM(in_features=3584, out_features=3584, bias=False, w_bit=4, group_size=128)
        )
        (mlp): Qwen2MLP(
          (gate_proj): WQLinear_GEMM(in_features=3584, out_features=18944, bias=False, w_bit=4, group_size=128)
          (up_proj): WQLinear_GEMM(in_features=3584, out_features=18944, bias=False, w_bit=4, group_size=128)
          (down_proj): WQLinear_GEMM(in_features=18944, out_features=3584, bias=False, w_bit=4, group_size=128)
          (act_fn)

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

prompt = "Tell me about O'Reilly online learning"
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
text

"<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\nTell me about O'Reilly online learning<|im_end|>\n<|im_start|>assistant\n"

In [5]:
import time

start = time.time()
print("\n\n*** Generate:")

tokens = tokenizer(text, return_tensors='pt')
output = model.generate(inputs=tokens.input_ids.cuda(), attention_mask=tokens.attention_mask.cuda(),
                        temperature=0.7, 
                        do_sample=True, top_p=0.95, top_k=40, 
                        max_new_tokens=512)
used = time.time() - start
tps = len(output[0]) / used
print(tokenizer.decode(output[0]))
print(f"{used} seconds, {tps} tokens/s")



*** Generate:
<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Tell me about O'Reilly online learning<|im_end|>
<|im_start|>assistant
O'Reilly Online Learning is part of O'Reilly Media, a well-known publisher of books and resources on technology, business, data science, software development, and more. O'Reilly Online Learning provides an extensive library of video courses, articles, and books that cater to both beginners and advanced learners in various fields.

Here are some key features and aspects of O'Reilly Online Learning:

1. **Video Courses**: O'Reilly offers hundreds of video courses taught by industry experts. These courses cover a wide range of topics, from programming languages (Python, JavaScript, etc.) to data analysis, machine learning, and design principles.

2. **Articles and Tutorials**: The platform includes a wealth of articles, tutorials, and practical guides that help you deepen your understanding and skills.

3. **Books**: Access to O'

In [ ]:
!nvidia-smi